# Библиотеки

In [1]:
import neptune
import pandas as pd

from cv import get_indices
from load_data import load
from model_selection import training
import random
from NN import simple_torchpl
from pl_framework import nn_training
from pytorch_forecasting.metrics import MAPE
import torch

In [2]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [3]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="3334104b-e4da-40ca-a353-d78790164c0b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

# Константы

In [4]:
data_path = 'data/'
random_state = 54321

# Загрузка данных

In [5]:
train_features, train_targets, _ = load(data_path)

# Валидация

In [6]:
cv = get_indices(train_targets, 
                 [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                  (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
                  (pd.to_datetime('2020-03-31 00:00:00'), pd.to_datetime('2020-04-15 00:00:00'))
                 ],
                 first_train = True
                )

In [7]:
#train_targets = train_targets.drop('timestamp', axis = 'columns')
#train_features = train_features.drop('timestamp', axis = 'columns')

# Подбор модели

## NN

In [ ]:
%%capture
for loss in ['MSE', 'MAE', 'MAPE']:
    for target in ['B_C2H6', 'B_C3H8', 'B_iC4H10', 'B_nC4H10']:
        my_training = training(name = target, nn_model = simple_torchpl, training_nn = nn_training,
                               description = 'Ffill. New train for each fold. Simple nn.2 Linears. 2 drops. loss: {}\
                               Separate shift for each feature. seq_len = 1.180-220 range for shift. No shift in rates\
                               No normalisation. Only first 2 fold and test set(last fold).\
                               Dropped data before 2020-02-15 00:00:00'.format(loss),
                               upload_source_files = ['cv.py', 
                                                      'load_data.py', 
                                                      'model_selection.py',
                                                      'NN.py',
                                                      'pl_framework.py',
                                                      'preprocessing.py'])
        my_training.set_up_studying(random_state = random_state)

        model = 'torch'

        def params_func(trial, X):
            return(
                {
                    'target': target,
                    'n_h_1': trial.suggest_int('n_h_1', 10, 1000),
                    'batch_size': trial.suggest_int('batch_size', 10, 800),
                    'p_1': trial.suggest_uniform('p_1', 0, 1),
                    'p_2': trial.suggest_uniform('p_2', 0, 1),
                    'lr': trial.suggest_loguniform('lr', 0.0001, 0.2),
                    'weight_decay': trial.suggest_uniform('weight_decay', 0.0001, 1),
                    'optimizer': 'AdamW',
                    'loss': loss,
                    'activation1': trial.suggest_categorical('activation1', ['Tanh','Hardtanh','Hardshrink', 'ELU' , 
                                                                             'SELU', 'ReLU', 'Tanhshrink', 'CELU']),
                    'n_back_A_CH4': trial.suggest_int('n_back_A_CH4', 180, 220),
                    'n_back_A_C2H6': trial.suggest_int('n_back_A_C2H6', 180, 220),
                    'n_back_A_C3H8': trial.suggest_int('n_back_A_C3H8', 180, 220),
                    'n_back_A_iC4H10': trial.suggest_int('n_back_A_iC4H10', 180, 220),
                    'n_back_A_iC5H12': trial.suggest_int('n_back_A_iC5H12', 180, 220),
                    'n_back_A_nC4H10': trial.suggest_int('n_back_A_nC4H10', 180, 220),
                    'n_back_A_nC5H12': trial.suggest_int('n_back_A_nC5H12', 180, 220),
                    'n_back_A_C6H14': trial.suggest_int('n_back_A_C6H14', 180, 220)

        }
            )


        n_trials = 30
        my_training.train(X = train_features, 
                          y = train_targets, 
                          cv = cv, 
                          model=model, 
                          params_func = params_func, 
                          n_trials = n_trials)
        neptune.stop()

## LGBM

In [ ]:
%%capture
for boosting in ['gbdt']:
    for objective in ['huber']:
        for target in ['B_C3H8']:
            my_training = training(name = target,
                                   description = 'Ffill. 1 train for each fold. LGBM. boosting = {}, objective = {}\
                                   Separate shift for each feature. seq_len = 1.180-220 range for shift. No shift in rates. features+features 1, 2, 3 ... n days ago\
                                   No normalisation. Only first 2 fold and test set(last fold).feature fraction = 1\
                                   Dropped data before 2020-02-15 00:00:00'.format(boosting, objective),
                                   upload_source_files = ['cv.py', 
                                                          'load_data.py', 
                                                          'model_selection.py',
                                                          'NN.py',
                                                          'pl_framework.py',
                                                          'preprocessing.py'])
            my_training.set_up_studying(random_state = random_state)

            model = 'lgbm'

            def params_func(trial, X):
                return(
                    {
                        'target': target,
                        'objective': objective,
                        'boosting': boosting,
                        'n_jobs': -1,
                        'n_estimators': 1,
                        'random_state': random_state,
                        'bagging_fraction': 1,
                        'feature_fraction': 1,
                        'n_today': trial.suggest_int('n_today', 0, 100),
                        'min_child_samples': trial.suggest_int('min_child_samples', 2, 256),
                        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
                        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 1.5),
                        'n_back_A_CH4': trial.suggest_int('n_back_A_CH4', 180, 220),
                        'n_back_A_C2H6': trial.suggest_int('n_back_A_C2H6', 180, 220),
                        'n_back_A_C3H8': trial.suggest_int('n_back_A_C3H8', 180, 220),
                        'n_back_A_iC4H10': trial.suggest_int('n_back_A_iC4H10', 180, 220),
                        'n_back_A_iC5H12': trial.suggest_int('n_back_A_iC5H12', 180, 220),
                        'n_back_A_nC4H10': trial.suggest_int('n_back_A_nC4H10', 180, 220),
                        'n_back_A_nC5H12': trial.suggest_int('n_back_A_nC5H12', 180, 220),
                        'n_back_A_C6H14': trial.suggest_int('n_back_A_C6H14', 180, 220)


            }
                )


            n_trials = 15000
            my_training.train(X = train_features, 
                              y = train_targets, 
                              cv = cv, 
                              model = model, 
                              params_func = params_func, 
                              n_trials = n_trials)
            neptune.stop()

## XGBOOST

In [ ]:
%%capture
for booster in ['gbtree']:
    for objective in ['reg:tweedie', 'reg:gamma', 'reg:pseudohubererror', 'reg:logistic', 'reg:squaredlogerror', 'reg:squarederror']:
        for target in ['B_C2H6', 'B_C3H8', 'B_iC4H10', 'B_nC4H10']:
            my_training = training(name = target,
                                   description = 'Ffill. 1 train for each fold. XGBOOST. boosting = {}, objective = {}\
                                   Separate shift for each feature. seq_len = 1.180-220 range for shift. No shift in rates\
                                   No normalisation. Only first 2 fold and test set(last fold).feature fraction = 1\
                                   Dropped data before 2020-02-15 00:00:00'.format(booster, objective),
                                   upload_source_files = ['cv.py', 
                                                          'load_data.py', 
                                                          'model_selection.py',
                                                          'NN.py',
                                                          'pl_framework.py',
                                                          'preprocessing.py'])
            my_training.set_up_studying(random_state = random_state)

            model = 'xgboost'

            def params_func(trial, X):
                return(
                    {
                        'target': target,
                        'objective': objective,
                        'booster': booster,
                        'nthread': -1,
                        'n_estimators': 3000,
                        'random_state': random_state,
                        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
                        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
                        'max_leaves': trial.suggest_int('max_leaves', 2, 256),
                        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 1.5),
                        'n_back_A_CH4': trial.suggest_int('n_back_A_CH4', 180, 220),
                        'n_back_A_C2H6': trial.suggest_int('n_back_A_C2H6', 180, 220),
                        'n_back_A_C3H8': trial.suggest_int('n_back_A_C3H8', 180, 220),
                        'n_back_A_iC4H10': trial.suggest_int('n_back_A_iC4H10', 180, 220),
                        'n_back_A_iC5H12': trial.suggest_int('n_back_A_iC5H12', 180, 220),
                        'n_back_A_nC4H10': trial.suggest_int('n_back_A_nC4H10', 180, 220),
                        'n_back_A_nC5H12': trial.suggest_int('n_back_A_nC5H12', 180, 220),
                        'n_back_A_C6H14': trial.suggest_int('n_back_A_C6H14', 180, 220)


            }
                )


            n_trials = 3000
            my_training.train(X = train_features, 
                              y = train_targets, 
                              cv = cv, 
                              model = model, 
                              params_func = params_func, 
                              n_trials = n_trials)
            neptune.stop()

## SKLEARN

In [8]:
from sklearn.linear_model import SGDRegressor

In [ ]:
%%capture
for target in ['C2H6', 'C3H8', 'iC4H10', 'nC4H10']:
    for random_state in [223, 245, 267, 12345, 998, 1456, 938, 223, 16]:
        my_training = training(name = 'B_' + target, sklearn_class = SGDRegressor,
                               description = 'Ffill. 1 train for each fold. Linear regression. WEIGHTED\
                               Separate shift for each feature. seq_len = 1.180-220 range for shift. No shift in rates. Features = A_target, constant\
                               No normalisation. Only first 2 fold and test set(last fold)\
                               Dropped data before 2020-02-15 00:00:00',
                               upload_source_files = ['cv.py', 
                                                      'load_data.py', 
                                                      'model_selection.py',
                                                      'NN.py',
                                                      'pl_framework.py',
                                                      'preprocessing.py'])
        my_training.set_up_studying(random_state = random_state)

        model = 'sklearn'

        def params_func(trial, X):
            return(
                {
                    'target': target,
                    'random_state': random_state,
                    'loss': 'epsilon_insensitive',
                    'epsilon': 0,
                    'tol': 1e-5,
                    'n_back_A_{}'.format(target): trial.suggest_int('n_back_A_{}'.format(target), 180, 220)


        }
            )


        n_trials = 250
        my_training.train(X = train_features, 
                          y = train_targets, 
                          cv = cv, 
                          model = model, 
                          params_func = params_func, 
                          n_trials = n_trials)
        neptune.stop()

[I 2020-12-12 20:03:45,075] A new study created in memory with name: no-name-bf1b92ad-276d-4ae7-8f30-52f6a8203f2b
[I 2020-12-12 20:03:45,426] Trial 0 finished with value: 0.06696306604099853 and parameters: {'n_back_A_C2H6': 210}. Best is trial 0 with value: 0.06696306604099853.
[I 2020-12-12 20:03:45,779] Trial 1 finished with value: 0.056542263282341854 and parameters: {'n_back_A_C2H6': 197}. Best is trial 1 with value: 0.056542263282341854.
[I 2020-12-12 20:03:45,815] Trial 2 finished with value: 0.06978198085353601 and parameters: {'n_back_A_C2H6': 182}. Best is trial 1 with value: 0.056542263282341854.
[I 2020-12-12 20:03:45,851] Trial 3 finished with value: 0.06696306604099853 and parameters: {'n_back_A_C2H6': 210}. Best is trial 1 with value: 0.056542263282341854.
[I 2020-12-12 20:03:45,885] Trial 4 finished with value: 0.08442863279894677 and parameters: {'n_back_A_C2H6': 219}. Best is trial 1 with value: 0.056542263282341854.
[I 2020-12-12 20:03:45,921] Trial 5 finished with v

Traceback (most recent call last):
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-2ff48eaaf3b2>", line 34, in <module>
    my_training.train(X = train_features,
  File "C:\Users\Sibmice\Desktop\Projects\sibur_2020\model_selection.py", line 55, in train
    self.study.optimize(lambda trial: self.objective(trial, X, y, cv, model, params_func),
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\study.py", line 306, in optimize
    _optimize(
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\_optimize.py", line 55, in _optimize
    _optimize_sequential(
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\_optimize.py", line 156, in _optimize_sequential
    trial = _run_trial(study, func, catch)
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\_optimize.py", line 189, in _run_trial
    value = func(trial)
  File

Traceback (most recent call last):
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-2ff48eaaf3b2>", line 34, in <module>
    my_training.train(X = train_features,
  File "C:\Users\Sibmice\Desktop\Projects\sibur_2020\model_selection.py", line 55, in train
    self.study.optimize(lambda trial: self.objective(trial, X, y, cv, model, params_func),
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\study.py", line 306, in optimize
    _optimize(
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\_optimize.py", line 55, in _optimize
    _optimize_sequential(
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\_optimize.py", line 156, in _optimize_sequential
    trial = _run_trial(study, func, catch)
  File "C:\Users\Sibmice\anaconda3\lib\site-packages\optuna\_optimize.py", line 189, in _run_trial
    value = func(trial)
  File